**Upload and Extract Dataset**

In [ ]:
!rm -rf hand_gesture_dataset


In [2]:
from zipfile import ZipFile

# Unzip the dataset
with ZipFile("hand_gesture_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("hand_gesture_dataset")


**Import necessary libraries**

In [3]:
! pip install tensorflow
! pip install keras

In [4]:
# Import necessary libraries
import os
import shutil
import random
import zipfile
from google.colab import files
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from sklearn.metrics import classification_report


**Organize Data into Train, Validation, and Test Sets**

In [5]:
# Paths
dataset_dir = 'hand_gesture_dataset/data'
train_dir = 'hand_gesture_dataset/train'
validation_dir = 'hand_gesture_dataset/validation'
test_dir = 'hand_gesture_dataset/test'

# Create train, validation, and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Gesture types
gestures = ['fist', 'thumbsup', 'thumbsdown', 'five']

# Split data and organize it into appropriate directories
for gesture in gestures:
    gesture_path = os.path.join(dataset_dir, gesture)
    images = os.listdir(gesture_path)

    # Shuffle images for randomness
    random.shuffle(images)

    # Calculate split points: 80% for training, 10% for validation, and 10% for test
    total_images = len(images)
    train_split = int(0.8 * total_images)
    val_split = int(0.1 * total_images)
    test_split = total_images - train_split - val_split

    # Use indices to split images accordingly
    train_images = images[:train_split]
    validation_images = images[train_split:train_split + val_split]
    test_images = images[train_split + val_split:]

    # Create necessary gesture folders in train, validation, and test directories
    os.makedirs(os.path.join(train_dir, gesture), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, gesture), exist_ok=True)
    os.makedirs(os.path.join(test_dir, gesture), exist_ok=True)

    # Copy train images
    for img in train_images:
        shutil.copy(os.path.join(gesture_path, img), os.path.join(train_dir, gesture, img))

    # Copy validation images
    for img in validation_images:
        shutil.copy(os.path.join(gesture_path, img), os.path.join(validation_dir, gesture, img))

    # Copy remaining images to test folder (with subfolder for each gesture)
    for img in test_images:
        shutil.copy(os.path.join(gesture_path, img), os.path.join(test_dir, gesture, img))

print("Data has been split into train, validation, and test sets with subfolders.")


Data has been split into train, validation, and test sets with subfolders.


**Loading Data Using ImageDataGenerator**

In [6]:
# Load and preprocess training and validation data using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the train, validation, and test data using flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
)


Found 5622 images belonging to 4 classes.
Found 701 images belonging to 4 classes.
Found 707 images belonging to 4 classes.


**Creating the Model**

In [7]:
# Define the CNN model
model = models.Sequential([
    layers.Input(shape=(48, 48, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


**Training the Model**

In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


175/175 ━━━━━━━━━━━━━━━━━━━━ 29s 154ms/step - accuracy: 0.7161 - loss: 0.6369 - val_accuracy: 1.0000 - val_loss: 0.0170
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - accuracy: 1.0000 - loss: 0.0211 - val_accuracy: 1.0000 - val_loss: 0.0113
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


175/175 ━━━━━━━━━━━━━━━━━━━━ 39s 144ms/step - accuracy: 0.9725 - loss: 0.0765 - val_accuracy: 0.9509 - val_loss: 0.0922
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 1.0000 - loss: 0.0274 - val_accuracy: 0.9655 - val_loss: 0.0522
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 40s 140ms/step - accuracy: 0.9892 - loss: 0.0467 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step - accuracy: 1.0000 - loss: 0.0235 - val_accuracy: 1.0000 - val_loss: 4.5381e-04
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 42s 147ms/step - accuracy: 0.9941 - loss: 0.0251 - val_accuracy: 1.0000 - val_loss: 0.0065
Epoch 8/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 9/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 41s 147ms/step - accuracy: 0.9907 - loss: 0.0340 - val_accuracy: 1.0000 - val_loss: 1.5877e-04
Epoch 10/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 277us/step - accuracy: 1.0000 - loss: 0.0039 

**Evaluating the Model**

In [9]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 1.0000 - loss: 1.7042e-04
Test accuracy: 1.00


**Class-wise accuracy**

In [10]:
# Get the predictions and true labels for the test set
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print the classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step
              precision    recall  f1-score   support

        fist       1.00      1.00      1.00       176
        five       1.00      1.00      1.00       203
  thumbsdown       1.00      1.00      1.00       165
    thumbsup       1.00      1.00      1.00       163

    accuracy                           1.00       707
   macro avg       1.00      1.00      1.00       707
weighted avg       1.00      1.00      1.00       707



**Saving the Model**

In [13]:
import json
import h5py

class_labels = list(train_generator.class_indices.keys())
labels_json = json.dumps(class_labels)

# Save the model
model.save('gesture_recognition_model.h5')

# Open the saved model file in append mode
with h5py.File('gesture_recognition_model.h5', mode='a') as f:
    f.attrs['class_labels'] = labels_json


**Making Predictions**

In [14]:
# Make predictions on the test set
predictions = model.predict(test_generator)

# Get predicted labels
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_generator.classes

# Compare the true and predicted labels
print(f"True Labels: {true_classes}")
print(f"Predicted Labels: {predicted_classes}")


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
True Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 